In [26]:
import numpy as np
import pandas as pd
import json

In [5]:
df1=pd.read_csv("E:/Model Data/Vehicle_data1.csv")
df2=pd.read_csv("E:/Model Data/Vehicle_data2.csv")
df3=pd.read_csv("E:/Model Data/Vehicle_data3.csv")

In [6]:
total_data=pd.concat([df1,df2,df3],ignore_index=True)

In [7]:
total_data.shape

(119142, 12)

In [8]:
total_data.head()

,HRLFC,TotalDistance,VehicleSpeed,EngineSpeed,AccPedalPosition,SweetSpot,TopGear,EngineOilPressure,EngineCoolantTemp,Distance_Travelled,Fuel_consumed,Fuel_Economy
0,50369.43,47840.32,13.09,1915.50,7.6,1,0,488,49,0.17,0.20,0.850000
1,50369.55,47840.48,0.00,598.75,0.0,7,0,312,55,0.16,0.12,1.333333
2,50369.82,47840.66,3.77,1487.75,38.0,7,0,424,63,0.18,0.27,0.666667
3,50369.86,47840.68,0.00,603.50,0.0,7,0,276,65,0.02,0.04,0.500000
4,50370.10,47840.75,7.05,1401.00,26.8,7,0,448,57,0.07,0.15,0.466667


In [9]:
total_data.isnull().sum()

HRLFC                 0
TotalDistance         0
VehicleSpeed          0
EngineSpeed           0
AccPedalPosition      0
SweetSpot             0
TopGear               0
EngineOilPressure     0
EngineCoolantTemp     0
Distance_Travelled    0
Fuel_consumed         0
Fuel_Economy          0
dtype: int64

In [10]:
total_data.describe()

,HRLFC,TotalDistance,VehicleSpeed,EngineSpeed,AccPedalPosition,SweetSpot,TopGear,EngineOilPressure,EngineCoolantTemp,Distance_Travelled,Fuel_consumed,Fuel_Economy
count,119142.000000,119142.000000,119142.000000,119142.000000,119142.000000,119142.000000,119142.000000,119142.000000,119142.000000,119142.000000,119142.000000,119142.000000
mean,54372.172550,52508.044179,14.091098,1303.061609,26.150577,4.567508,0.008989,319.630659,87.391600,0.241854,0.190376,1.968366
std,2300.564827,2754.379313,12.699499,507.542639,25.839133,2.622819,0.094385,95.006256,5.591088,0.205358,0.172944,3.100202
min,50369.430000,47840.320000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.010000,0.037037
25%,52371.050000,50218.430000,1.510000,781.620000,0.000000,1.000000,0.000000,244.000000,85.000000,0.050000,0.060000,0.500000
50%,54396.180000,52338.875000,12.590000,1383.120000,22.000000,6.000000,0.000000,352.000000,88.000000,0.210000,0.120000,1.000000
75%,56327.630000,54799.660000,23.160000,1692.750000,44.400000,7.000000,0.000000,392.000000,90.000000,0.380000,0.280000,2.153846
max,58338.610000,57465.940000,51.860000,2824.000000,100.000000,7.000000,1.000000,1020.000000,100.000000,2.090000,2.450000,60.000000


In [11]:
fuel_economy_95th_percentile = total_data['Fuel_Economy'].quantile(0.95)
total_data['Fuel_Economy'] = np.where(total_data['Fuel_Economy'] > fuel_economy_95th_percentile, fuel_economy_95th_percentile, total_data['Fuel_Economy'])
total_data['Fuel_Economy'].describe()

count    119142.000000
mean          1.691803
std           1.737042
min           0.037037
25%           0.500000
50%           1.000000
75%           2.153846
max           6.500000
Name: Fuel_Economy, dtype: float64

In [12]:
total_data=total_data[['VehicleSpeed', 'EngineSpeed','AccPedalPosition','EngineOilPressure','EngineCoolantTemp','Fuel_Economy']]

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [14]:
x=total_data[['VehicleSpeed', 'EngineSpeed','AccPedalPosition','EngineOilPressure','EngineCoolantTemp']]
y=total_data['Fuel_Economy']

In [15]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [16]:
model=RandomForestRegressor()
model.fit(x_train,y_train)

RandomForestRegressor()

In [17]:
y_pred = model.predict(x_test)

In [18]:
r2=r2_score(y_test,y_pred)
r2

0.8399594392123684

In [19]:
efficient_threshold = total_data['Fuel_Economy'].quantile(0.50)

In [20]:
json.dump(efficient_threshold,open("efficient_threshold.json","w"))

In [21]:
def classify_fuel_economy(fuel_economy, efficient_threshold):
    if fuel_economy >= efficient_threshold:
        return 'Efficient'
    else:
        return 'Inefficient'

In [22]:
def predict_and_classify(current_driving_data, model, efficient_threshold):
    predicted_fuel_economy = model.predict([current_driving_data])[0]
    classification = classify_fuel_economy(predicted_fuel_economy, efficient_threshold)
    return classification

In [29]:
import pickle
with open('fuel_economy_model.pkl', 'wb') as file:
    pickle.dump(model, file)

In [30]:
with open('efficient_threshold.pkl', 'wb') as file:
    pickle.dump(efficient_threshold, file)